# Практичне 2. Булевий пошук


По заданій колекції (10 документів по 150К) документів побудувати:

* матрицю інцидентності "термін-документ"
* інвертований індекс

Обгрунтуйте обрані структури збереження даних в розрізі їх ефективності при збільшенні об'ємів даних.

Порівняти розмір структур, що вийшли.

Зробити булевий пошук по цим структурам (по обом).

Оператори: `AND`, `OR`, `NOT`. Формат в запиті на власний розсуд

## Ініціалізуємо бібліотеку `nltk`

In [3]:
from info_search.lab_1.nltk_utils import init_nltk

init_nltk()

[nltk_data] Downloading package punkt to /home/o.kyba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Читаємо словник

In [2]:
from info_search.lab_1.lexicon import Lexicon

with open("data/lexicon.protobuf", "rb") as f:
    lexicon = Lexicon.from_proto(f.read())

print(f"Кількість термінів у словнику: {len(lexicon.terms)}")

Кількість термінів у словнику: 36447


## Матриця інцедентності

### Будуємо матрицю інцидентності

Клас `info_search.lab_2.incidence_matrix.IncidenceMatrixBuilder` виконє побудову матриці інцидентності. Структрура даних для неї це масив масивів `int8` бібілотеки `nunpy`.

In [6]:
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

from info_search.lab_1.parsers import parse_terms
from info_search.lab_1.preprocessing import WordsNormalizer, WordsTokenizer
from info_search.lab_1.readers import EpubReader
from info_search.lab_2.incidence_matrix import IncidenceMatrixBuilder

books = (
    "books/451-za-farengeytom.epub",
    "books/1984.epub",
    "books/atlant-rozpraviv-pliechi-1.epub",
    "books/atlant-rozpraviv-pliechi-2.epub",
    "books/atlant-rozpraviv-pliechi-3.epub",
    "books/haksli-oldos-prekrasnyy-novyy-svit.epub",
    "books/kulbabove-vino.epub",
    "books/na-zakhidnomu-fronti-bez-zmin.epub",
    "books/proshchavai-zbroie.epub",
    "books/sapients-istoriya-lyudstva.epub",
)

reader = EpubReader(books)
tokenizer = WordsTokenizer(stopwords.words("ukrainian"))
normalizer = WordsNormalizer(MorphAnalyzer(lang="uk"))
matrix_builder = IncidenceMatrixBuilder()

for doc_name, term in parse_terms(reader, tokenizer, normalizer):
    term_id = lexicon.get_term_id(term)
    doc_id = lexicon.get_doc_id(doc_name)
    matrix_builder.add_term(doc_id, term_id)

matrix_index = matrix_builder.build()

print(matrix_index.data_struct)

  0%|          | 0/10 [00:00<?, ?it/s]/home/o.kyba/PycharmProjects/info_search/venv/lib/python3.11/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
100%|██████████| 10/10 [00:50<00:00,  5.05s/it]

[[1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


### Внутрішня будова

І хоча ми використовуємо для матриці інцидентності масиви з бібілотеки `numpy`, що є більш оптимізовані ніж `list`, така структура даних не є ефеутивною для зберігання. Кожен новий термін та документ значно збільшують розмірність матриці. 

In [7]:
import sys

print(f"Розмірність матриці: {matrix_index.data_struct.shape}")
print(f"Розмір структури даних у пам'яті: {sys.getsizeof(matrix_index.data_struct)}")

Розмірність матриці: (36447, 291)
Розмір структури даних у пам'яті: 10606205


## Булевий пошук

Пошук був реалізований за допомгою бібліотеки `luqum`, яка виконує парсинг запиту у форматі `lucene` та дає зручні інструменти по обходу синтаксичного дерева.

Усі булеві операції виконуються над масивами за допомогою бібілотеки `numpy`, що відповідають за наявності терміну у  документах.

Виконаємо булевий пошук для такого запиту: `міністерство OR (Таґґарт AND NOT Джеймс)`.

In [8]:
from info_search.lab_2.incidence_matrix import IncidenceMatrixQueryExecutor

matrix_query_exec = IncidenceMatrixQueryExecutor(lexicon, matrix_index, normalizer)
matrix_docs = matrix_query_exec.execute("міністерство OR (Таґґарт AND NOT Джеймс)")
print(matrix_docs)

['1984.epub/Chapter001_1.html', '1984.epub/Chapter001_2.html', '1984.epub/Chapter001_3.html', '1984.epub/Chapter001_4.html', '1984.epub/Chapter001_5.html', 'atlant-rozpraviv-pliechi-1.epub/ch1-16.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-19.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-21.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-3.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-8.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-4.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-6.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-12.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-18.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-24.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-27.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-26.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-8.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch

## Інвертований індекс

### Будуємо інверотваний індекс

Клас info_search.lab_2.inverted_index.InvertedIndexBuilder виконє побудову інвертованого ідексу. Структрура даних для нього - це хеш-таблиця, ключ ідентифікатор терміну, а знаяення є відсортованим списком ідентифікаторів документів (`dict[int, list[int]`).


In [3]:
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

from info_search.lab_1.parsers import parse_terms
from info_search.lab_1.preprocessing import WordsNormalizer, WordsTokenizer
from info_search.lab_1.readers import EpubReader
from info_search.lab_2.inverted_index import InvertedIndexBuilder

books = (
    "books/451-za-farengeytom.epub",
    "books/1984.epub",
    "books/atlant-rozpraviv-pliechi-1.epub",
    "books/atlant-rozpraviv-pliechi-2.epub",
    "books/atlant-rozpraviv-pliechi-3.epub",
    "books/haksli-oldos-prekrasnyy-novyy-svit.epub",
    "books/kulbabove-vino.epub",
    "books/na-zakhidnomu-fronti-bez-zmin.epub",
    "books/proshchavai-zbroie.epub",
    "books/sapients-istoriya-lyudstva.epub",
)

reader = EpubReader(books)
tokenizer = WordsTokenizer(stopwords.words("ukrainian"))
normalizer = WordsNormalizer(MorphAnalyzer(lang="uk"))
index_builder = InvertedIndexBuilder()

for doc_name, term in parse_terms(reader, tokenizer, normalizer):
    term_id = lexicon.get_term_id(term)
    doc_id = lexicon.get_doc_id(doc_name)
    index_builder.add_term(doc_id, term_id)

inverted_index = index_builder.build()

print(inverted_index.data_struct)

100%|██████████| 10/10 [00:49<00:00,  4.99s/it]

{0: [0, 148, 178], 1: [0, 1, 4, 7, 8, 60, 67, 70, 75, 78, 81, 82, 102, 104, 112, 144, 156, 178, 216, 261], 2: [0, 1, 6, 7, 8, 145], 3: [0, 1, 3, 47, 49, 60, 65, 69, 102, 149, 153, 271, 288], 4: [0, 1, 7, 46, 47, 49, 50, 59, 65, 72, 118, 136, 146, 273, 277, 278, 284, 286, 288], 5: [0, 1, 6, 9, 10, 15, 27, 35, 42, 49, 57, 64, 70, 73, 78, 106, 122, 132, 137, 146, 159, 168, 176, 188, 191, 193, 199, 216, 221, 271, 277, 280, 283, 288], 6: [0], 7: [0], 8: [0, 7, 46, 55, 61, 62, 64, 67, 68, 76, 81, 82, 83, 86, 87, 88, 90, 91, 92, 94, 95, 96, 98, 100, 102, 105, 107, 108, 109, 112, 115, 116, 120, 121, 122, 123, 124, 126, 127, 128, 130, 131, 132, 136, 138, 140, 141, 142, 144, 145, 146, 152, 175, 190], 9: [0, 1, 3, 4, 5, 6, 7, 46, 47, 48, 49, 50, 53, 54, 55, 56, 57, 59, 60, 61, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 1

### Внутрішня будова

Структура даних для інверотваного індекса є більш вдалою, ніж для матриці інцидентності. При додаванні нового терміну на треба додати в хеш-таблицю лише список ідентифікаторів документів, де він з'являється. При додаванні ж нового документу, нам треба за знайти всі терміни та вставити ідентифікатор для відсортованих списків, що можна ефективно зробити навіть звмчайним двійковим пошуком. Отже, така струкутра даних має бути економнішою, ніж матриця інцидентності, при збільшені кількості даних.

In [4]:
import sys

print(f"Розмір структури даних у пам'яті: {sys.getsizeof(inverted_index.data_struct)}")

Розмір структури даних у пам'яті: 1310800


### Булевий пошук

Реалізувати операції булевого пошуку для інфвертованого індексу трохи складніше. Ми маємо ітреруватися бо двом відсортованим спискам одночасно і, залежно від порівняння елементів, додавати чи не додавати їх у результуючий список.

In [5]:
from info_search.lab_2.inverted_index import InvertedIndexQueryExecutor

index_query_exec = InvertedIndexQueryExecutor(lexicon, inverted_index, normalizer)
index_docs = index_query_exec.execute("міністерство OR (Таґґарт AND NOT Джеймс)")
print(index_docs)

['1984.epub/Chapter001_1.html', '1984.epub/Chapter001_2.html', '1984.epub/Chapter001_3.html', '1984.epub/Chapter001_4.html', '1984.epub/Chapter001_5.html', 'atlant-rozpraviv-pliechi-1.epub/ch1-16.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-19.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-21.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-3.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-8.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-4.xhtml', 'atlant-rozpraviv-pliechi-1.epub/ch1-6.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-12.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-18.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-24.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-27.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-26.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-25.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-1.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch1-8.xhtml', 'atlant-rozpraviv-pliechi-2.epub/ch

## Порівняння результатів

In [9]:
print(
    "Чи однкові результати обробки запиту матрицею інцидентності"
    f"та інвертованим індексом? {matrix_docs == index_docs}"
)

Чи однкові результати обробки запиту матрицею інцидентностіта інвертованим індексом? True


Резульати для матриці інцидентності та інфверотваного індексу однакові. Це означає, що запити виконуються коректно, або хочаб з однаковими помилкаим 😂